In [1]:
import MDAnalysis as mda
import numpy as np
import os
from scipy.optimize import minimize
from utils import *

c:\Users\gbonn\anaconda3\envs\md-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# read in each of the substrate files
directory = 'substrate_files/'
file_names = [f for f in os.listdir(directory) if "_head" not in f]

for curr_file_name in file_names:
    file_start = curr_file_name.split('.')[0]
    # load substrate universe
    substrate = mda.Universe(directory+curr_file_name)
    # identify the atoms that comprise the aka substrates 
    substrate_important_indexes = get_substrate_aka_indexes(substrate)
    selected_atoms = substrate.atoms[list(substrate_important_indexes.values())]
    selected_atoms.write(directory + file_start + "_head.pdb")


c:\Users\gbonn\anaconda3\envs\md-env\Lib\site-packages\MDAnalysis\coordinates\PDB.py:777: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
c:\Users\gbonn\anaconda3\envs\md-env\Lib\site-packages\MDAnalysis\coordinates\PDB.py:1153: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"
c:\Users\gbonn\anaconda3\envs\md-env\Lib\site-packages\MDAnalysis\coordinates\PDB.py:1200: UserWarning: Found missing chainIDs. Corresponding atoms will use value of 'X'
  warnings.warn("Found missing chainIDs."


In [3]:
substrate_6 = mda.Universe(directory + '6.pdb')
substrate_6_important_indexes = get_substrate_aka_indexes(substrate_6)
print(substrate_6_important_indexes)

receptor = mda.Universe('int1_receptor.pdb')
ThDP_important_indexes = get_ThDP_indexes(receptor)
print(ThDP_important_indexes)

{'C2': 1, 'O1': 12, 'C3': 0, 'O2': 11, 'O3': 9, 'R': 2}
{'C1': 1, 'N1': 2, 'S1': 0, 'N2': 20}


In [4]:
def atom_objective(point, centers, radii):
    return sum((np.linalg.norm(point - center) - radius)**2 for center, radius in zip(centers, radii))

def combined_objective(all_points, centers, radii):
    # Split all_points into the three key points
    C2, C3, O1 = np.split(all_points, len(all_points)/3)
    
    # Calculate atom-level errors for each point to fit them to sphere constraints
    atom_err_C2 = atom_objective(C2, centers, radii[0])
    atom_err_C3 = atom_objective(C3, centers, radii[1])
    atom_err_O1 = atom_objective(O1, centers, radii[2])
    
    total_atom_err = atom_err_C2 + atom_err_C3 + atom_err_O1
    
    # Combine errors with weighting factors
    return  total_atom_err 

def optimize_points(centers, initial_guess, radii):
    # Flatten initial guess as midpoint of each set of centers
    # Set up optimization
    tolerance = 1e-6
    result = minimize(
        combined_objective,
        initial_guess,
        args=(centers, radii),
        tol=tolerance
    )
    C2, C3, O1 = np.split(result.x, 3)
    # Check for successful optimization
    if result.success or result.fun < tolerance:
        print('CONVERGED')
    else:
        print('NOT CONVERGED')
    return C2, C3, O1

In [5]:
tpp_residue = receptor.select_atoms("resname TPP")

C1_coords = get_atom_position(tpp_residue,ThDP_important_indexes['C1'])
N1_coords = get_atom_position(tpp_residue,ThDP_important_indexes['N1'])
N2_coords = get_atom_position(tpp_residue,ThDP_important_indexes['N2'])
S1_coords = get_atom_position(tpp_residue,ThDP_important_indexes['S1'])

vector_S1_to_C1 = C1_coords - S1_coords
vector_N1_to_C1 = C1_coords - N1_coords
avg_vector = (vector_S1_to_C1 + vector_N1_to_C1)/2
unit_vector = avg_vector / np.linalg.norm(avg_vector)
guess_C2 = C1_coords + unit_vector * 1.54


print(guess_C2)

[-34.66339  -36.011883  22.165665]


In [6]:
# Example input
# ThDP C1, N1, N2, S1 atom coords
centers = np.array([C1_coords,N1_coords,N2_coords,S1_coords])
print(centers)
# radii are in order of (columns) C1, N1, N2, S1 and then rows (C2, C3, O1) 
radii = [
    [1.539,2.562,3.389,2.880],
    [2.533,3.205,4.764,3.784],
    [2.393,2.973,2.592,3.893]
]
#initial_guess = np.hstack([np.mean(centers, axis=0) for i in range(3)])
initial_guess = np.hstack([guess_C2 for i in range(3)])
#print(init)
C2_optimized, C3_optimized, O1_optimized = optimize_points(centers, initial_guess, radii)
all_optimized = [C2_optimized, C3_optimized, O1_optimized]
print("Optimized points:", C2_optimized, C3_optimized, O1_optimized)

C2_err = atom_objective(C2_optimized, centers, radii[0])
C3_err = atom_objective(C3_optimized, centers, radii[1])
O1_err = atom_objective(O1_optimized, centers, radii[2])

all_errors = [C2_err,C3_err,O1_err]
min_error_index = all_errors.index(min(all_errors))
redo_initial_guess = np.hstack([all_optimized[min_error_index] for i in range(3)])

C2_reoptimized, C3_reoptimized, O1_reoptimized = optimize_points(centers, redo_initial_guess, radii)
print("Reoptimized points:", C2_reoptimized, C3_reoptimized, O1_reoptimized)


[[-34.894 -35.257  23.488]
 [-33.783 -34.924  24.16 ]
 [-33.487 -33.076  21.57 ]
 [-36.272 -34.716  24.347]]
NOT CONVERGED
Optimized points: [-34.95322067 -36.05735187  22.17123387] [-34.57322144 -37.59150683  22.56317159] [-34.04892146 -35.57772853  21.2669495 ]
CONVERGED
Reoptimized points: [-34.94014025 -36.07518795  22.18527961] [-34.57235262 -37.60205263  22.58615327] [-34.04679877 -35.58955324  21.27459049]


In [10]:
initial_positions = [get_atom_position(substrate_6,substrate_6_important_indexes['C2']),get_atom_position(substrate_6,substrate_6_important_indexes['C3']),get_atom_position(substrate_6,substrate_6_important_indexes['O1'])]
final_positions = [C2_reoptimized, C3_reoptimized, O1_reoptimized]

In [11]:
R, t = kabsch_algorithm(initial_positions,final_positions)

In [33]:
t

array([-37.90549569, -33.33337875,  23.36411089])

In [12]:
transformed_coords = []
for atom_coords in substrate_6.atoms.positions:
    new_coords = np.dot(R, atom_coords) + t
    transformed_coords.append(new_coords)

substrate_6.atoms.positions = transformed_coords

# Save the updated universe to a new PDB file
output_filename = "aligned_substrate_6.pdb"
substrate_6.atoms.write(output_filename)

In [21]:
C2_coords = substrate_6.atoms.positions[substrate_6_important_indexes['C2']]
O1_coords = substrate_6.atoms.positions[substrate_6_important_indexes['O1']]
C3_coords = substrate_6.atoms.positions[substrate_6_important_indexes['C3']]
R_coords = substrate_6.atoms.positions[substrate_6_important_indexes['R']]

In [16]:
def get_angle(coord1,coord2,coord3):
        # Calculate vectors BA and BC
        BA = coord1 - coord2
        BC = coord3 - coord2

        # Compute the dot product and magnitudes of BA and BC
        dot_product = np.dot(BA, BC)
        magnitude_BA = np.linalg.norm(BA)
        magnitude_BC = np.linalg.norm(BC)

        # Calculate the cosine of the angle
        cos_theta = dot_product / (magnitude_BA * magnitude_BC)

        # Handle potential floating-point errors
        cos_theta = np.clip(cos_theta, -1.0, 1.0)

        # Calculate the angle in radians and then convert to degrees
        angle_radians = np.arccos(cos_theta)
        angle_degrees = np.degrees(angle_radians)

        return angle_degrees


In [ ]:
C1_coords
N1_coords
N2_coords
S1_coords

In [34]:
def angle_objective(atom1,atom2,atom3,angle):
    angle_err = abs(get_angle(atom1,atom2,atom3) - angle)
    return angle_err

def combined_angle_objective(tail_coord, C1_coord, C2_coord, O1_coord, C3_coord, angles):
    # Calculate atom-level errors for each point to fit them to sphere constraints
    angle_err_C1_C2_R = angle_objective(C1_coord, C2_coord, tail_coord, angles[0])
    angle_err_O1_C2_R = angle_objective(O1_coord, C2_coord, tail_coord, angles[1])
    angle_err_C3_C2_R = angle_objective(C3_coord, C2_coord, tail_coord, angles[2])
    
    #dist_err = get_dist()

    total_angle_err = angle_err_C1_C2_R + angle_err_O1_C2_R + angle_err_C3_C2_R
    # Combine errors with weighting factors
    return  total_angle_err 

def optimize_angles(angles,initial_guess):
    # Flatten initial guess as midpoint of each set of centers
    # Set up optimization
    tolerance = 1e-6
    max_dist = 1.382
    bounds = [(C2_coords[0]-max_dist,C2_coords[0]+max_dist),(C2_coords[1]-max_dist,C2_coords[1]+max_dist),(C2_coords[2]-max_dist,C2_coords[2]+max_dist)]
    result = minimize(
        combined_angle_objective,
        initial_guess,
        args=(C1_coords,C2_coords,O1_coords,C3_coords,angles),
        tol=tolerance,
        bounds=bounds
    )
    
    # Check for successful optimization
    if result.success or result.fun < tolerance:
        print('CONVERGED')
    else:
        print('NOT CONVERGED')
    
    return result.x

In [36]:
R_coord_guess = optimize_angles([111.1,110.2,107.5],R_coords)

NOT CONVERGED


In [37]:
t_R = R_coord_guess-R_coords

In [ ]:

transformed_coords = []
for atom_coords in substrate_6.atoms.positions:
    new_coords = atom_coords + t_R
    transformed_coords.append(new_coords)

substrate_6.atoms.positions = transformed_coords

# Save the updated universe to a new PDB file
output_filename = "aligned_substrate_6.pdb"
substrate_6.atoms.write(output_filename)